### Code definitions

In [1]:
import os
import sys
from pathlib import Path
from asgiref.sync import sync_to_async
import string
import pandas as  pd
from datetime import date
import yfinance as yf
import sqlite3

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'share_dinkum_proj.settings')

import django
django.setup()

from django.conf import settings
import share_dinkum_app.models as app_models

from share_dinkum_app import loading

import logging
logger = logging.getLogger(__name__)

In [2]:
import_data_folder = settings.BASE_DIR / 'share_dinkum_app' / 'import_data'

personal_data = import_data_folder / 'data_import_template_personal.xlsx'
sample_data = import_data_folder / 'data_import_template_public.xlsx'


defaults = {
    'sample_data' : {
        'default_name' : 'sample_data',
        'default_email' : 'sample_data@example.com',
        'password' : 'password',
        'default_portfolio_description' : 'Sample Data Portfolio',
        'input_file' : sample_data,
    },
    'own_data' : {
        'default_name' : 'admin',
        'default_email' : 'admin@example.com',
        'password' : 'password',
        'default_portfolio_description' : 'Default Portfolio',
        'input_file' : personal_data,
    },
}


In [3]:
@sync_to_async
def add_account(selection='sample_data'):

    default_name = defaults[selection]['default_name']
    default_email = defaults[selection]['default_email']
    password = defaults[selection]['password']
    default_portfolio_description = defaults[selection]['default_portfolio_description']

    
    fiscal_year_type, created = app_models.FiscalYearType.objects.get_or_create(
        description='Australian Tax Year',
        defaults={'start_month': 7, 'start_day': 1}
    )

    if not app_models.AppUser.objects.filter(username=default_name).exists():
        user = app_models.AppUser.objects.create_superuser(username=default_name, email=default_email, password=password)   # Please change this password after logging in
        logger.info("Superuser created successfully!")
    owner = app_models.AppUser.objects.get(username=default_name)

    record = {'description': default_portfolio_description, 'owner': owner, 'fiscal_year_type': fiscal_year_type}
    account, created = app_models.Account.objects.get_or_create(description=default_portfolio_description, defaults=record)
    return account

@sync_to_async
def clear_all_data():
    loading.DataLoader.clear_all_data()
    try:
        del account # To avoid stale references
    except NameError:
        pass

@sync_to_async
def load_all_data(account, input_file):
    loading.DataLoader(account=account, input_file=input_file)

@sync_to_async
def load_historical_prices(df, account):
    loader = loading.DataLoader(account=account, input_file=None)
    loader.load_table_to_model(model=app_models.InstrumentPriceHistory, df=df)
    
@sync_to_async
def update_all_price_history(account):
    account.update_all_price_history()

In [4]:
await clear_all_data()

2025-08-23 14:28:23 [ERROR] share_dinkum_app.loading: Error deleting model Instrument: Missing currency value
Traceback (most recent call last):
  File "c:\code\share-dinkum\share_dinkum_proj\share_dinkum_app\loading.py", line 359, in clear_all_data
    model.objects.all().delete()  # Deletes all records in the model
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\code\share-dinkum\.venv\Lib\site-packages\django\db\models\query.py", line 1198, in delete
    collector.collect(del_query)
    ~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "c:\code\share-dinkum\.venv\Lib\site-packages\django\db\models\deletion.py", line 281, in collect
    new_objs = self.add(
        objs, source, nullable, reverse_dependency=reverse_dependency
    )
  File "c:\code\share-dinkum\.venv\Lib\site-packages\django\db\models\deletion.py", line 124, in add
    if not objs:
           ^^^^
  File "c:\code\share-dinkum\.venv\Lib\site-packages\django\db\models\query.py", line 398, in __bool__
    self._fetch_all()
    ~~~~~~~~~~~

ValueError: Missing currency value

In [ ]:
selection = 'sample_data'

if defaults['own_data']['input_file'].exists():
    res = input("Personal import file exists. Do you want to use it? (y/n): ")
    if res.upper() == 'Y':
        selection = 'own_data'
        logger.info('Using own data')
    else:
        selection = 'sample_data'
        logger.info('Using generic data')


input_file = defaults[selection]['input_file']

account = await add_account(selection=selection)

await load_all_data(account=account, input_file=input_file)

Using own data
Superuser created successfully!
Loading Market


100%|██████████| 11/11 [00:00<00:00, 184.41it/s]


Loading Instrument


100%|██████████| 45/45 [00:00<00:00, 108.38it/s]


Loading Buy


ValueError: Missing currency value

In [ ]:
from djmoney.money import Money

1  + None

In [ ]:
# You can ignore this cell

legacy_db = Path(r'C:\code\finance-database\finance-database.db')

if selection == 'own_data' and legacy_db.exists():
    query = "SELECT * FROM [price_history] where ticker_code != 'AUDUSD=X'"
    parameters = None
    with sqlite3.connect(legacy_db) as conn:
        df = pd.read_sql_query(query, conn, params=parameters)
    df['account'] = account
    df['instrument__name'] = df['ticker_code'].apply(lambda x : x.split('.')[0])
    df['date'] = df['date'].apply(lambda x : date.fromisoformat(x))
    df = df.drop(columns=['id', 'ticker_code', 'capital_gains', 'dividends'])
    df = df[~df['high'].isna()]

    
    await load_historical_prices(df=df, account=account)

In [ ]:
await update_all_price_history(account=account)